In [1]:
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import APSIMGraphHelpers as AGH
import GraphHelpers as GH
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
import matplotlib.dates as mdates
import MathsUtilities as MUte
import shlex # package to construct the git command to subprocess format
import subprocess 
import xmltodict, json
import sqlite3
from scipy.optimize import basinhopping
from scipy.optimize import dual_annealing
from skopt import gp_minimize
%matplotlib inline

In [2]:
con = sqlite3.connect(r'C:\GitHubRepos\ApsimX\Tests\Validation\Wheat\Wheat.db')
DailyReport = pd.read_sql("Select * from DailyReport",con)

In [3]:
Simulations = pd.read_sql("Select * from _Simulations",con)
Simulations.set_index('ID',inplace=True)

In [43]:
Observed = pd.read_sql("Select * from Observed",con).dropna(axis=1,how='all')
Observed.loc[:,'SimulationName'] = [Simulations.loc[x,'Name'] for x in Observed.SimulationID]
Observed.set_index(['SimulationName','Clock.Today'],drop=False,inplace=True)
Observed.sort_index(inplace=True)
Observed.sort_index(inplace=True,axis=1)

In [67]:
HarvestPred = pd.read_sql("Select * from HarvestReport",con).dropna(axis=1,how='all')
HarvestPred.loc[:,'SimulationName'] = [Simulations.loc[x,'Name'] for x in HarvestPred.SimulationID]
HarvestPred.set_index(['SimulationName','Clock.Today'],drop=False,inplace=True)
HarvestPred.sort_index(inplace=True)
HarvestPred.sort_index(inplace=True,axis=1)

In [37]:
Factors = pd.read_sql("Select * from _Factors",
                    con)
Factors.set_index('SimulationID',drop=False,inplace=True)
Factors = Factors.sort_values(by=['FactorName']).drop_duplicates()
Factors.sort_index(inplace=True)
Factors.sort_index(inplace=True,axis=1)

In [6]:
Simulations

,Name,FolderName
ID,,
1,APS26NRate160WaterWet,SE Queensland
2,APS26NRate40WaterDry,SE Queensland
3,APS26NRate0WaterWet,SE Queensland
4,APS26NRate160WaterDry,SE Queensland
5,APS26NRate40WaterWet,SE Queensland
...,...,...
7960,FAR WAE W22-02MgmtGrazedCvAccroc,
7961,FAR WAE W22-02MgmtHigh InputCvRockstar,
7962,FAR WAE W22-02MgmtHigh InputCvMowhawk,


In [7]:
Factors

,CheckpointID,ExperimentName,FactorName,FactorValue,FolderName,SimulationID
SimulationID,,,,,,
1,1,APS26,Water,Wet,SE Queensland,1
1,1,APS26,NRate,160,SE Queensland,1
2,1,APS26,Water,Dry,SE Queensland,2
2,1,APS26,NRate,40,SE Queensland,2
3,1,APS26,Water,Wet,SE Queensland,3
...,...,...,...,...,...,...
6796,1,PotentialGrainSize,Value,Hartog40,ProteinAccumulation,6796
6797,1,PotentialGrainSize,Value,Hartog44,ProteinAccumulation,6797
6799,1,TerminalWaterStress,Irrigation,Full,TerminalWaterStress,6799


In [38]:
list(Observed.loc[Observed.loc[:,'Wheat.Phenology.CurrentStageName']=='HarvestRipe',:].dropna(axis=1,how='all').columns)

['([Wheat].Leaf.Transpiration + [Soil].SoilWater.Es + [MicroClimate].PrecipitationInterception)',
 'CheckpointID',
 'Clock.Today',
 'NDVIModel.Script.NDVI',
 'NDVIModel.Script.NDVI.se',
 'SimulationID',
 'SimulationName',
 'Soil.Water.Volumetric(1)',
 'Soil.Water.Volumetric(2)',
 'Soil.Water.Volumetric(3)',
 'Soil.Water.Volumetric(4)',
 'Soil.Water.Volumetric(5)',
 'Soil.Water.Volumetric(6)',
 'Soil.Water.Volumetric(7)',
 'Soil.Water.Volumetric(8)',
 'Wheat.AboveGround.N',
 'Wheat.AboveGround.NError',
 'Wheat.AboveGround.Nconc',
 'Wheat.AboveGround.Nconc.se',
 'Wheat.AboveGround.Wt',
 'Wheat.AboveGround.Wt.se',
 'Wheat.AboveGround.WtError',
 'Wheat.DaysAfterSowing',
 'Wheat.Ear.N',
 'Wheat.Ear.Nconc',
 'Wheat.Ear.Wt',
 'Wheat.Grain.Density',
 'Wheat.Grain.Density.se',
 'Wheat.Grain.FWt',
 'Wheat.Grain.FWt15',
 'Wheat.Grain.Moisture',
 'Wheat.Grain.Moisture.se',
 'Wheat.Grain.N',
 'Wheat.Grain.NConc',
 'Wheat.Grain.NError',
 'Wheat.Grain.Number',
 'Wheat.Grain.NumberError',
 'Wheat.Grai

In [69]:
HarvestObs = Observed.loc[Observed.loc[:,'Wheat.Phenology.CurrentStageName'] == 'HarvestRipe',:].copy()
HarvestObs.dropna(axis=1,how='all',inplace=True)
HarvestObs.dropna(axis=0,how='all',inplace=True)

In [80]:
Simulations.loc[1,'Name']

'APS26NRate160WaterWet'

In [91]:
Observed.loc['Lincoln2014IrrigFull','Wheat.AboveGround.Wt'].dropna()

C:\Users\Cflhxb\AppData\Local\Temp\ipykernel_27852\3135578429.py:1: PerformanceWarning: indexing past lexsort depth may impact performance.
  Observed.loc['Lincoln2014IrrigFull','Wheat.AboveGround.Wt'].dropna()


Clock.Today
2013-07-04 12:00:00     259.608929
2013-09-03 12:00:00     649.678571
2013-10-22 12:00:00    1402.830746
2013-11-04 12:00:00    1751.334901
2013-11-18 12:00:00    1887.006123
2013-12-02 12:00:00    2249.084598
2013-12-17 12:00:00    2993.458720
2014-01-25 12:00:00    2625.494371
Name: Wheat.AboveGround.Wt, dtype: float64

In [93]:
Simulations

,Name,FolderName
ID,,
1,APS26NRate160WaterWet,SE Queensland
2,APS26NRate40WaterDry,SE Queensland
3,APS26NRate0WaterWet,SE Queensland
4,APS26NRate160WaterDry,SE Queensland
5,APS26NRate40WaterWet,SE Queensland
...,...,...
7960,FAR WAE W22-02MgmtGrazedCvAccroc,
7961,FAR WAE W22-02MgmtHigh InputCvRockstar,
7962,FAR WAE W22-02MgmtHigh InputCvMowhawk,


In [102]:
for sim in ['Lincoln2014IrrigFull']:#Simulations.index:
    name = sim#Simulations.loc[sim,'Name']
    obsDat = pd.DataFrame()
    try:
        obsDat = Observed.loc[name,:].copy()
    except:
        print('no Obs Data for '+name)
    obsDat.dropna(axis=1,how='all',inplace=True)
    obsDat.dropna(axis=0,how='all',inplace=True)

In [103]:
obsDat

,CheckpointID,Clock.Today,SimulationID,SimulationName,Soil.Water.Volumetric(1),Soil.Water.Volumetric(2),Soil.Water.Volumetric(3),Soil.Water.Volumetric(4),Soil.Water.Volumetric(5),Soil.Water.Volumetric(6),...,Wheat.Leaf.SpecificAreaCanopy,Wheat.Leaf.StemPopulation,Wheat.Leaf.Tips,Wheat.Phenology.CurrentStageName,Wheat.Phenology.Zadok.Stage,Wheat.Population,Wheat.SowingData.Cultivar,Wheat.Spike.Wt,Wheat.Stem.Wt,sum([Soil].Water.MM)
Clock.Today,,,,,,,,,,,,,,,,,,,,,
2013-04-05 12:00:00,1,2013-04-05 12:00:00,728,Lincoln2014IrrigFull,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,NaN,NaN,Wakanui,NaN,NaN,NaN
2013-04-16 12:00:00,1,2013-04-16 12:00:00,728,Lincoln2014IrrigFull,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,NaN,NaN,Wakanui,NaN,NaN,NaN
2013-04-22 12:00:00,1,2013-04-22 12:00:00,728,Lincoln2014IrrigFull,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,3.8,None,17.5,NaN,Wakanui,NaN,NaN,NaN
2013-04-23 12:00:00,1,2013-04-23 12:00:00,728,Lincoln2014IrrigFull,0.270625,0.27825,0.2760,0.25025,0.25075,0.23975,...,NaN,NaN,NaN,None,NaN,NaN,Wakanui,NaN,NaN,390.225
2013-04-26 12:00:00,1,2013-04-26 12:00:00,728,Lincoln2014IrrigFull,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,NaN,NaN,Wakanui,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-01-21 12:00:00,1,2014-01-21 12:00:00,728,Lincoln2014IrrigFull,0.073500,0.18625,0.1985,0.15550,0.18325,0.19125,...,NaN,NaN,NaN,None,NaN,NaN,Wakanui,NaN,NaN,263.050
2014-01-23 12:00:00,1,2014-01-23 12:00:00,728,Lincoln2014IrrigFull,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,None,93.0,NaN,Wakanui,NaN,NaN,NaN
2014-01-25 12:00:00,1,2014-01-25 12:00:00,728,Lincoln2014IrrigFull,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,HarvestRipe,NaN,NaN,Wakanui,347.640077,883.743502,NaN
